<a href="https://www.kaggle.com/code/sakiiib/fake-news-lstm?scriptVersionId=194092114" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [3]:
df=pd.read_csv('/kaggle/input/fake-news/train.csv')
df=df.dropna()

In [4]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [5]:
X=df.drop('label', axis=1)

In [6]:
y=df['label']

In [7]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [8]:
voc_size=5000
messages=X.copy()

In [9]:
messages.reset_index(inplace=True)

In [10]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]


In [13]:
sent_length=20
embedded_docs= pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [14]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3769,
       3343, 1661, 3338, 2222, 4894, 3519, 1669, 4114, 4134], dtype=int32)

In [15]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [17]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [18]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x_final, y_final, test_size=.33, random_state=42)


In [20]:
model1.fit( x_train, y_train, validation_data=( x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.7799 - loss: 0.4273 - val_accuracy: 0.9173 - val_loss: 0.1977
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9477 - loss: 0.1381 - val_accuracy: 0.9248 - val_loss: 0.1850
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.9664 - loss: 0.0920 - val_accuracy: 0.9198 - val_loss: 0.2426
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9810 - loss: 0.0558 - val_accuracy: 0.9190 - val_loss: 0.2786
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.9839 - loss: 0.0452 - val_accuracy: 0.9157 - val_loss: 0.2646
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9917 - loss: 0.0263 - val_accuracy: 0.9138 - val_loss: 0.3448
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9944 - loss: 0.0190 - val_accuracy: 0.9188 - val_loss: 0.4710
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9936 - loss: 0.0181 - val

In [21]:
ypred= model1.predict(x_test)
y_pred=[ypred>.5]
y_pred=y_pred[0]

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [23]:
cm= confusion_matrix(y_test, y_pred)
cm

array([[3129,  290],
       [ 246, 2370]])

In [24]:
accuracy_score(y_test, y_pred)

0.9111847555923778

In [25]:
print(classification_report( y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

